## Load data

In [21]:
import sys
sys.path.append('/users/kuba/code/aic/mil')

import pandas
import torch
import numpy

from torch.utils.data import DataLoader, SubsetRandomSampler

from sklearn import model_selection



data = pandas.read_csv('data.csv').values
ids = pandas.read_csv('ids.csv', squeeze = True).values
labels = pandas.read_csv('labels.csv', squeeze = True).values

data = torch.tensor(data)
ids = torch.tensor(ids)
labels = torch.tensor(labels)

# Create dataset
dataset = mil.MilDataset(data, ids, labels, normalize = True)

# Create train and test data loaders
indices = numpy.arange(len(dataset))
train_indices, test_indices = model_selection.train_test_split(indices, shuffle = True, test_size = 0.2)

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dl = DataLoader(dataset, sampler = train_sampler, batch_size = len(train_indices), collate_fn=mil.collate)
test_dl = DataLoader(dataset, sampler = test_sampler, batch_size = len(test_indices), collate_fn=mil.collate)

## Define model, criterion and optimizer

In [27]:
import mil_pytorch.mil as mil

n_neurons1 = 10
n_neurons2 = 10
input_len = len(dataset.data[0])

prepNN = torch.nn.Sequential(
    torch.nn.Linear(input_len, n_neurons1, bias = True),
    torch.nn.ReLU(),
)

afterNN = torch.nn.Sequential(
    torch.nn.Linear(n_neurons2, 1),
    torch.nn.Tanh()
)

model = mil.BagModel(prepNN, afterNN, aggregation_func = torch.mean)

criterion = mil.MyHingeLoss()

learning_rate = 1e-4
weight_decay = 1e-6

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)

## Train model

In [ ]:
import time

epochs = 4000

start = time.time()
print('TRAINING:')

train_losses = torch.empty(0)

for epoch in range(epochs):
    
    # Optimization
    for data, info, labels in train_dl:
        pred = model((data, info))
        loss = criterion(pred[:,0], labels)

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses = torch.cat((train_losses, loss.float()))

    train_loss = torch.mean(train_losses, dim = 0, keepdim = True)

    train_losses = torch.new_empty(0)



    # Print message
    if (epoch+1)%100 == 0:
        print('[{}/{}] | train_loss: {} | valid_loss: {} '.format(epoch+1, epochs, train_loss.item(), valid_loss.item()))

print('INFO: Finished training - elapsed time: {}'.format(time.time() - start))

In [1]:
import torch

In [7]:
a = torch.Tensor([1.2, 5.6])
b = torch.Tensor([2.3])

In [8]:
b.to(a.device)

tensor([2.3000])

In [10]:
b.device

device(type='cpu')